In [115]:
import torch
import numpy as np
import pandas as pd

In [116]:
data = pd.read_csv('processed.cleveland.data.csv', sep=',')
import scipy.stats as stats
independentVars = data.keys()[:-1]

data = data[data['iv12'] != '?']
data = data[data['iv13'] != '?']

data = data.astype({'iv12': "float64", 'iv13': 'float64', 'dv1': "float64"})

data[independentVars] = data[independentVars].apply(stats.zscore)

In [118]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [119]:
y[np.where(y > 0)] = 1
y = y.reshape((-1, 1))

In [120]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, shuffle=True, train_size=0.8)
train_set = torch.utils.data.TensorDataset(torch.tensor(train_X, dtype=torch.float), torch.tensor(train_y, dtype=torch.float))
test_set = torch.utils.data.TensorDataset(torch.tensor(test_X, dtype=torch.float), torch.tensor(test_y, dtype=torch.float))


In [121]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=4, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=len(test_set.tensors[0]))

In [139]:
import torch.nn as nn
HeartDiseaseClassifier = nn.Sequential(
    nn.Linear(13, 30),
    nn.ReLU(),
    nn.Linear(30, 30),
    nn.ReLU(),
    nn.Linear(30, 1)
)

lossFun = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=HeartDiseaseClassifier.parameters(), lr=.001)

for epoch_i in range(1000):
  for X,y in train_loader:
    preds = HeartDiseaseClassifier(X)
    loss = lossFun(preds, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [140]:
X_test, y_test = next(iter(test_loader))
test_preds = HeartDiseaseClassifier(X_test)
test_preds[test_preds <= 0.00] = 0.00
test_preds[test_preds > 0] = 1.00
acc = (torch.mean((test_preds == y_test).float())*100.00).item()
print(f"{acc} % accuracy on test set")

76.66666412353516 % accuracy on test set


In [144]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test.detach().numpy(), test_preds.detach().numpy()).ravel()
(tn, fp, fn, tp)

(28, 5, 9, 18)